# Lakehouse Agent - Deploy Athena Database

This notebook deploys the Athena database and tables for the lakehouse data layer.

**What this notebook does:**
- Uses the S3 bucket created in prerequisites setup
- Uploads sample claims and users data to S3
- Creates Athena database: `lakehouse_db`
- Creates tables: `claims` and `users`
- Verifies deployment with test queries

**Prerequisites:**
- ✅ Completed `00-prerequisites-setup.ipynb` (S3 bucket must be created)
- ✅ SSM parameters configured with `/app/lakehouse-agent/` prefix
- ✅ AWS credentials with Athena, S3, and Glue permissions

**IAM Permissions Required:**
- `athena:*`
- `s3:*`
- `glue:*`
- `ssm:GetParameter`

**Duration:** ~10 minutes

In [ ]:
# AWS Initialization - Load credentials and create session
from utils.notebook_init import init_aws
import subprocess
from pathlib import Path

# This will:
# 1. Load credentials from .env file (if it exists)
# 2. Create and validate AWS session (env vars take precedence over SSO)
# 3. Return session, region, and account_id for use in this notebook
session, region, account_id = init_aws()

print(f"✅ Ready to proceed with AWS operations")
print(f"   Account ID: {account_id}")
print(f"   Region: {region}")

## Step 1: Validate Prerequisites

Check that all required SSM parameters from the previous notebook exist.

In [ ]:
# Initialize AWS clients using the validated session from cell 1
ssm_client = session.client('ssm', region_name=region)

# Check required parameters with new naming convention
print("🔍 Validating prerequisites...\n")

required_params = [
    '/app/lakehouse-agent/s3-bucket-name',
    '/app/lakehouse-agent/database-name',
    '/app/lakehouse-agent/athena-workgroup'
]

missing = []
config_values = {}

for param in required_params:
    try:
        response = ssm_client.get_parameter(Name=param)
        value = response['Parameter']['Value']
        # Extract key name for config_values dict
        key = param.split('/')[-1]
        config_values[key] = value
        print(f"✅ {param}: {value}")
    except ssm_client.exceptions.ParameterNotFound:
        print(f"❌ {param}: NOT FOUND")
        missing.append(param)

if missing:
    print(f"\n❌ Missing parameters: {', '.join(missing)}")
    print("Please run 00-prerequisites-setup.ipynb first")
else:
    print("\n✅ All prerequisites validated!")
    
    # Load configuration from SSM
    BUCKET_NAME = config_values['s3-bucket-name']
    DATABASE_NAME = config_values['database-name']
    WORKGROUP = config_values.get('athena-workgroup', 'primary')
    
    print(f"\n📋 Configuration:")
    print(f"   Bucket: {BUCKET_NAME}")
    print(f"   Database: {DATABASE_NAME}")
    print(f"   Workgroup: {WORKGROUP}")

## Step 2: Deploy Athena Database

Run the Athena setup script to create the database, tables, and upload sample data.

**Note**: The S3 bucket was already created in the prerequisites notebook. This step will use that existing bucket and create Athena tables on top of it.

In [ ]:
print("🚀 Running Athena setup...\n")

print(f"📦 Using S3 bucket from SSM: {BUCKET_NAME}")
print(f"   The setup script will read this from SSM Parameter Store")
print()

# Run setup_athena.py WITHOUT --bucket-name argument
# The script will automatically read the bucket name from SSM Parameter Store
result = subprocess.run(
    ['python', 'setup_athena.py'],
    cwd='deployment/athena-setup',
    capture_output=True,
    text=True
)

print(result.stdout)

if result.returncode != 0:
    print("❌ Error during Athena setup:")
    print(result.stderr)
else:
    print("\n✅ Athena setup completed successfully!")
    print("\n💾 Database and tables created using existing S3 bucket")

## Step 3: Validate Deployment

Verify that the database and tables were created successfully.

In [ ]:
print("🔍 Validating Athena deployment...\n")

# Use session from cell 1 to create AWS clients
athena_client = session.client('athena', region_name=region)
glue_client = session.client('glue', region_name=region)

# Check database exists
try:
    response = glue_client.get_database(Name=DATABASE_NAME)
    print(f"✅ Database '{DATABASE_NAME}' exists")
except glue_client.exceptions.EntityNotFoundException:
    print(f"❌ Database '{DATABASE_NAME}' not found")

# Check tables exist
try:
    response = glue_client.get_tables(DatabaseName=DATABASE_NAME)
    tables = response['TableList']
    
    print(f"\n📋 Tables in {DATABASE_NAME}:")
    for table in tables:
        table_name = table['Name']
        column_count = len(table['StorageDescriptor']['Columns'])
        print(f"   • {table_name} ({column_count} columns)")
    
    if len(tables) >= 2:
        print("\n✅ All tables created successfully")
    else:
        print(f"\n⚠️  Expected 2 tables, found {len(tables)}")
        
except Exception as e:
    print(f"❌ Error checking tables: {e}")

## Next Steps

✅ **Athena Database Deployment Complete!**

Your Athena database is now set up with:
- Database: `lakehouse_db`
- Tables: `claims` (9 sample claims), `users` (3 test users)
- S3 data location: `s3://{BUCKET_NAME}/lakehouse-data/`

**Next:** Run `02-deploy-cognito.ipynb` to set up authentication.

### Test Queries

You can test the deployment with these queries in the Athena console:

```sql
-- Count all claims
SELECT COUNT(*) as total_claims FROM lakehouse_db.claims;

-- View claims for user001
SELECT claim_id, claim_type, claim_status, claim_amount 
FROM lakehouse_db.claims 
WHERE user_id = 'user001@example.com';

-- View all users
SELECT * FROM lakehouse_db.users;
```

### Verify SSM Parameters

The setup script automatically saves configuration to SSM:
```bash
aws ssm get-parameter --name /app/lakehouse-agent/s3-bucket-name
aws ssm get-parameter --name /app/lakehouse-agent/database-name
```